In [1]:
# Libraries Installations
import warnings
warnings.filterwarnings('ignore')

agent_packages = {
    "autogen": "pyautogen",
    "autogen_core" :"autogen-core==0.4.0.dev9",
    "autogen.agentchat": "autogen-agentchat==0.4.0.dev9",
    "autogen_ext": "autogen-ext[openai]==0.4.0.dev9",
    'openai':'openai',
    'azure.identity':'azure_identity',
    "pandas_datareader": "pandas_datareader",
    "nest_asyncio": "nest_asyncio",
}
# Create a for loop to install each package only if it is not already installed
for package_import, package_install in agent_packages.items():
    try:
        __import__(package_import)
        print(f"{package_import} is already installed.")
    except ImportError:
        print(f"Installing {package_install}...")
        %pip install {package_install} -U

autogen is already installed.
autogen_core is already installed.
autogen.agentchat is already installed.
autogen_ext is already installed.
openai is already installed.
azure.identity is already installed.
pandas_datareader is already installed.
nest_asyncio is already installed.


In [2]:
from datetime import datetime
import openai
from openai import AzureOpenAI
from autogen_ext.models import AzureOpenAIChatCompletionClient
from autogen_core.components.models import UserMessage
from google.colab import userdata

In [3]:
# Azure OpenAI API Key-based authentication
ChatGPT4Endpoint = 'https://azopenai-demo.openai.azure.com/'
ChatGPT4_OPENAI_API_VERSION = '2024-05-01-preview'
GPT4DeploymentName = "gpt35-demo"
API_KEY =  userdata.get('AZ_OPENAI_KEY')

# Using the autogen extension with key-based authentication
clientGPT4 = AzureOpenAIChatCompletionClient(
    model='gpt-4-32k',
    azure_endpoint=ChatGPT4Endpoint,
    api_key=API_KEY,
    api_version=ChatGPT4_OPENAI_API_VERSION,
    azure_deployment=GPT4DeploymentName
)

## Group chat with planning and code execution

Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

**Note**: Here, we'll use an alternative method of code execution by providing a dict config. However, we can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [4]:
config_list_GPT4 = {
    "model": "gpt-4-32k",
    "api_type": "azure",
    "api_version": ChatGPT4_OPENAI_API_VERSION,
    "azure_endpoint": ChatGPT4Endpoint,
    "azure_deployment": GPT4DeploymentName,
    "api_key": API_KEY  # Replace with your actual Azure OpenAI API key
}
config_list = config_list_GPT4
model_client = clientGPT4

In [5]:
import autogen
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=config_list,
    human_input_mode="NEVER",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message= """Given a task, please determine what information is needed to complete the task.
                       Please note that the information will all be retrieved using Python code.
                       Please only suggest information that can be retrieved using Python code.
                       After each step is done by others, check the progress and instruct the remaining
                       steps. If a step fails, try to workaround""",
    description=""" Given a task, determine what information is needed to complete the task. After each
                    step is done by others, check the progress and instruct the remaining steps""",
    llm_config=config_list,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=config_list,
    description="An engineer that writes code based on the plan provided by the planner.",
)

executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=config_list,
    system_message="""Please write blogs in markdown format (with relevant titles) and put the content in pseudo
                      ```md``` code block. You take feedback from the admin and refine your blog.""",
    description="""Write blogs based on the code execution results and take feedback from the admin to refine the blog."""
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=config_list
)

In [9]:
today = datetime.now().date()
task = f"Write a blogpost about the fundamental analysis of a stock price performance. Today's date is {today}. Use Yahoo finance for the stock data if required"

# print("Using task: ", task)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the fundamental analysis of a stock price performance. Today's date is 2025-01-12. Use Yahoo finance for the stock data if required

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To write a blog post about fundamental analysis of a stock price performance, we will need the following information:

1. Historical price data of the stock: We can retrieve the historical price data using the yfinance library in Python. We will need the stock symbol and the date range for which we want the data.

2. Financial statements of the company: We can use web scraping libraries like BeautifulSoup or Selenium to extract financial statements (such as income statement, balance sheet, and cash flow statement) from reliable financial websites like Yahoo Finance or SEC EDGAR.

3. Company's industry and sector information: We can retrieve the in

In [12]:
# groupchat_result.summary

# Analyzing Stock Price Performance Through Fundamental Analysis

*Date: 2025-01-12*

## Introduction

When it comes to investing in the stock market, there are two main approaches: fundamental analysis and technical analysis. While technical analysis focuses on trends and patterns in stock prices, fundamental analysis delves into the financial health and performance of a company. In this blog post, we will explore the concept of fundamental analysis, its importance in stock market investing, and how it differs from technical analysis.

## Step 1: Collect Historical Price Data

To perform a thorough fundamental analysis of a stock's price performance, it is essential to analyze its historical price data. Historical price data provides insights into a stock's past performance and helps us identify trends, patterns, and crucial turning points that can aid in making informed investment decisions. One way to retrieve historical price data is by utilizing the `yfinance` library in Python.

Here is a step-by-step guide to collecting historical price data using the `yfinance` library:

1. Install the `yfinance` library using pip:

```
pip install yfinance
```

2. Import the library and specify the stock symbol and date range:

```python
import yfinance as yf

# Specify the stock symbol
symbol = "AAPL"

# Specify the start and end date of the historical data
start_date = "2015-01-01"
end_date = "2024-12-31"

# Retrieve the historical price data
stock_data = yf.download(symbol, start=start_date, end=end_date)
```

3. Analyze the retrieved historical price data using various visualization and statistical techniques to gain insights into the stock's performance.

## Step 2: Assess Financial Statements

Financial statements, such as the income statement, balance sheet, and cash flow statement, offer valuable insights into a company's financial health. Analyzing these statements helps us assess the company's profitability, liquidity, and overall financial stability. It enables us to evaluate various financial ratios, such as the price-to-earnings ratio, debt-to-equity ratio, and return on equity, which aid in determining the company's valuation and growth potential.

To obtain financial statements, one can use web scraping techniques to extract the necessary information from reliable financial websites such as Yahoo Finance or SEC EDGAR. Libraries like BeautifulSoup or Selenium can be utilized for web scraping.

Here is an example of using BeautifulSoup to scrape financial statements from Yahoo Finance:

```python
import requests
from bs4 import BeautifulSoup

# Specify the URL of the company's financial statements on Yahoo Finance
url = "https://finance.yahoo.com/quote/AAPL/financials?p=AAPL"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Extract the required financial statements data from the parsed HTML
# (Code for extracting the data)

# Perform analysis on the extracted financial statements data
# (Code for analysis)
```

## Step 3: Evaluate Industry and Sector Information

To gain a comprehensive understanding of a stock's performance, it is crucial to consider the industry and sector in which the company operates. Evaluating industry-specific data helps us assess how the company's stock performs relative to its peers and competitors. This analysis provides insights into the broader market conditions and identifies potential growth opportunities or risks within the sector.

Industry and sector information can be sourced from various financial websites or using the `yfinance` library, which provides details about the company's industry classification.

Here is an example of using the `yfinance` library to obtain the industry and sector information for a stock:

```python
import yfinance as yf

# Specify the stock symbol
symbol = "AAPL"

# Create a ticker object for the stock
ticker = yf.Ticker(symbol)

# Retrieve the industry and sector information
industry = ticker.info["industry"]
sector = ticker.info["sector"]

print("Industry:", industry)
print("Sector:", sector)
```

## Step 4: Analyze Dividend and Earnings Data

Dividend and earnings data play a crucial role in evaluating a company's financial performance and attractiveness for income-focused investors. Analyzing dividends helps us determine if the company has a history of consistent and growing dividend payments. Examining earnings data helps us assess the company's profitability over time.

Dividend and earnings data can be obtained from financial websites such as Yahoo Finance using web scraping techniques to extract the necessary information.

Here is an example of using web scraping to retrieve dividend and earnings data from Yahoo Finance:

```python
import requests
from bs4 import BeautifulSoup

# Specify the URL of the company's dividend and earnings data on Yahoo Finance
url = "https://finance.yahoo.com/quote/AAPL/history?p=AAPL"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Extract the required dividend and earnings data from the parsed HTML
# (Code for extracting the data)

# Perform analysis on the extracted dividend and earnings data
# (Code for analysis)
```

## Step 5: Consider Macroeconomic Indicators

Understanding a stock's performance in the broader economic context requires considering macroeconomic indicators. These indicators, such as GDP growth rates, interest rates, inflation rates, or industry-specific data, help assess the potential risks and opportunities for a stock investment.

Macroeconomic indicators can be collected from financial websites or government agencies using web scraping techniques or data APIs.

## Step 6: Apply Fundamental Analysis Techniques

Once you have collected all the relevant data, you can apply various fundamental analysis techniques to evaluate the stock's price performance. Here are a few commonly used techniques:

- **Financial Ratio Analysis**: Compare various financial ratios derived from the company's financial statements (e.g., price-to-earnings ratio, debt-to-equity ratio) to industry benchmarks or historical data. This helps assess the company's valuation and growth potential.

- **Valuation Models**: Utilize valuation models like discounted cash flow (DCF) analysis or price-to-earnings (P/E) ratio analysis to gain insights into the stock's intrinsic value and potential future returns.

- **Peer and Industry Comparison**: Compare the company's financial metrics, growth rates, and overall performance against its peers or industry benchmarks. This helps identify trends, strengths, and weaknesses in the company's performance.

By applying these fundamental analysis techniques, you can make informed investment decisions based on the stock's price performance potential and its underlying financial health.

## Conclusion

Fundamental analysis is a valuable approach for analyzing stock price performance. By considering historical price data, financial statements, industry and sector information, dividend and earnings data, as well as macroeconomic indicators, you can gain a comprehensive understanding of a stock's potential. Applying fundamental analysis techniques allows you to make informed investment decisions based on the company's financial health, valuation, and industry position.

*Disclaimer: The information provided in this blog post is for educational purposes only. It should not be considered as financial advice. Please consult with a professional financial advisor before making any investment decisions.*

## References

- Placeholder for references to be added.

In [11]:
import pprint
pprint.pprint(groupchat_result.cost)

{'usage_excluding_cached_inference': {'gpt-35-turbo-16k': {'completion_tokens': 1010,
                                                           'cost': 0.020213000000000002,
                                                           'prompt_tokens': 5391,
                                                           'total_tokens': 6401},
                                      'total_cost': 0.020213000000000002},
 'usage_including_cached_inference': {'gpt-35-turbo-16k': {'completion_tokens': 1010,
                                                           'cost': 0.020213000000000002,
                                                           'prompt_tokens': 5391,
                                                           'total_tokens': 6401},
                                      'total_cost': 0.020213000000000002}}
